In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape

In [ ]:
positive_path = "/content/drive/MyDrive/Laboratory/Positive"
negative_path = "/content/drive/MyDrive/Laboratory/Negative"

positive_images = [os.path.join(positive_path, img) for img in os.listdir(positive_path)]
negative_images = [os.path.join(negative_path, img) for img in os.listdir(negative_path)]

In [ ]:
images = []
labels = []

for img_path in positive_images:
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))  # Resize image to fit MobileNetV2 input size
    images.append(img)
    labels.append(1)  # Positive class

for img_path in negative_images:
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))  # Resize image to fit MobileNetV2 input size
    images.append(img)
    labels.append(0)  # Negative class

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Conv2D(4, kernel_size=(3, 3), padding='same')(x)  # 4 bounding box coordinates
x = Reshape((-1, 4))(x)
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(np.array(X_test))
    # Convert predictions to binary (0 or 1) based on confidence threshold
    binary_predictions = np.argmax(predictions, axis=2)

    def calculate_accuracy(predictions, labels):
        # Convert labels to binary array
        labels_binary = np.array(labels)
        labels_binary = np.expand_dims(labels_binary, axis=1)
        labels_binary = np.tile(labels_binary, (1, predictions.shape[1]))

        # Flatten predictions and labels
        predictions_flat = predictions.flatten()
        labels_flat = labels_binary.flatten()

        correct_predictions = np.sum(predictions_flat == labels_flat)
        accuracy = correct_predictions / len(labels_flat)
        return accuracy

    def calculate_precision(predictions, labels):
        true_positives = np.sum(np.logical_and(predictions == 1, labels == 1))
        predicted_positives = np.sum(predictions == 1)
        precision = true_positives / predicted_positives
        return precision

    def calculate_recall(predictions, labels):
        true_positives = np.sum(np.logical_and(predictions == 1, labels == 1))
        actual_positives = np.sum(labels == 1)
        recall = true_positives / actual_positives
        return recall

    accuracy = calculate_accuracy(binary_predictions, y_test)
    precision = calculate_precision(binary_predictions, y_test)
    recall = calculate_recall(binary_predictions, y_test)

    return accuracy, precision, recall


In [ ]:
accuracy, precision, recall = evaluate_model(model, X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')

3/3 [==============================] - 5s 1s/step
Accuracy: 43.18%
Precision: 0.00%
Recall: nan%


<ipython-input-9-550c33fc7853>:29: RuntimeWarning: invalid value encountered in scalar divide
  recall = true_positives / actual_positives


In [ ]:
print(len(positive_images))
print(len(negative_images))

240
240


In [ ]:
# Ensure the number of positive and negative images matches
min_images = min(len(positive_images), len(negative_images))
positive_images = positive_images[:min_images]
negative_images = negative_images[:min_images]

# Generate predictions
predictions_binary = np.argmax(model.predict(np.array(images)), axis=2)
predictions_labels = ["Weapon" if pred == 1 else "No Weapon" for pred in predictions_binary.flatten()]

# Create DataFrames for positive and negative images
positive_df = pd.DataFrame({'Image_Path': positive_images,
                            'Actual_Label': 'Weapon',
                            'Predicted_Label': predictions_labels[:min_images]})
negative_df = pd.DataFrame({'Image_Path': negative_images,
                            'Actual_Label': 'No Weapon',
                            'Predicted_Label': predictions_labels[:min_images]})  # Use the same indices as positive for negative

# Concatenate DataFrames
predictions = pd.concat([positive_df, negative_df], ignore_index=True)

# Save DataFrame to CSV
predictions.to_csv('/content/predictions_with_predictions.csv', index=False)


15/15 [==============================] - 20s 1s/step


In [ ]:
from google.colab import files
files.download('predictions_with_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>